In [1]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import json
# %load_ext line_profiler

In [2]:
client = MongoClient('localhost')
db = client.Returns_WealthNFunds

In [3]:
def insertMongo(collectionName,data):
    db.get_collection(collectionName).drop()
    db.get_collection(collectionName).insert_many(json.loads(data.reset_index().to_json(orient='records',date_format='iso')))

def readMongo(collectionName):
    data  = pd.DataFrame(list(db.get_collection(collectionName).find()))
#     data[colDate] = pd.to_datetime(data[colDate])
#     data.set_index(colDate,inplace=True)
    return data

In [4]:
rawPrices = readMongo('rawPrices').set_index('Date').drop(columns=['_id'])
dividend = readMongo('dividend').set_index('Date').drop(columns=['_id'])
splitRatio = readMongo('splitRatio').set_index('Date').drop(columns=['_id'])

In [106]:
# rawPrices=pd.read_csv('excel/output/rawPrices.csv',index_col='Date')
# dividend=pd.read_csv('excel/output/dividend.csv',index_col='Date')
# splitRatio=pd.read_csv('excel/output/splitRatio.csv',index_col='Date')

In [8]:
def calcAdjustedPrices(rawPrices,dividend,splitRatio):
    rawPrices.sort_index(ascending=False, inplace=True)
    dividend.sort_index(ascending=False, inplace=True)
    splitRatio.sort_index(ascending=False, inplace=True)

    pivot_adj_price_df = pd.DataFrame(index=rawPrices.index,data = np.zeros(rawPrices.shape),columns=rawPrices.columns)
    pivot_adj_price_df.iloc[0] = round(rawPrices.iloc[0],4)

    for i in range(1,len(rawPrices)):
        pivot_adj_price_df.iloc[i] = (pivot_adj_price_df.iloc[i - 1].values *
                       (((rawPrices.iloc[i].values * splitRatio.iloc[i - 1].values) - dividend.iloc[i - 1].values) / rawPrices.iloc[i - 1].values))

#     adjPrice = pivot_adj_price_df.stack().reset_index().rename(columns={0:'adjPrice','level_1':'ISIN'})

    return pivot_adj_price_df

In [9]:
pivotAdjPriceDf = calcAdjustedPrices(rawPrices,dividend,splitRatio)
insertMongo("adjustedPrice", pivotAdjPriceDf)

In [38]:
pivotAdjPriceDf.to_csv('excel/output/pivotAdjustedPrices.csv')
adjPrice.to_csv('excel/output/adjustedPrices.csv')

In [ ]:
# with pd.option_context('display.max_rows',None):
#     print(adjPrice.pivot_table(index='Date',columns='ISIN',values='adjPrice'))

In [ ]:
# TEST CASES
testISIN = 'INE042A01014'
pd.DataFrame(data = {'adj':adjPriceDf[testISIN],'raw':rawPrices[testISIN],'dividend':dividend[testISIN],'split':splitRatio[testISIN]}).to_csv('excel/testcases/test_{}.csv'.format(testISIN))